In [1]:
import numpy as np
import random

# keras
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
#from keras.layers import Concatenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

np.random.seed(13)

In [2]:
import autokeras as ak
import keras.backend as K

In [3]:
import sys
sys.version

'3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]'

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
%matplotlib inline
sns.set()
sns.set_style('whitegrid')

In [34]:
import pickle

In [5]:
tf_df = pd.read_csv('../train_features.csv',index_col='sig_id')

In [6]:
tts_df = pd.read_csv('../train_targets_scored.csv',index_col='sig_id')
moa1 = tts_df.iloc[:,0]

In [7]:
print(tf_df.shape, tts_df.shape)

(23814, 875) (23814, 206)


In [8]:
tf_df.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [10]:
872/8

109.0

In [12]:
input_layer = Input(shape=(872,))
hidden_layer = Dense(109, activation='relu')(input_layer)
output_layer = Dense(2, activation='softmax')(hidden_layer)

model_proto = Model(inputs=input_layer, outputs=output_layer)
model_proto.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 872)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 109)               95157     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 220       
Total params: 95,377
Trainable params: 95,377
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_proto.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', 
               metrics=['acc','ce'])

In [14]:
23814/42

567.0

In [16]:
model_proto.fit(tf_df.iloc[:,3:], tts_df.iloc[:,0], epochs=3, batch_size=42)

Epoch 1/3
567/567 [==============================] - 2s 3ms/step - loss: 0.0320 - acc: 0.9933 - ce: 0.0320
Epoch 2/3
567/567 [==============================] - 1s 3ms/step - loss: 0.0072 - acc: 0.9993 - ce: 0.0072
Epoch 3/3
567/567 [==============================] - 1s 3ms/step - loss: 0.0062 - acc: 0.9993 - ce: 0.0062


In [17]:
preds1 = model_proto.predict(tf_df.iloc[:,3:])
len(preds1[preds1==1])

1804

In [20]:
type(preds1)

numpy.ndarray

In [21]:
preds1.shape

(23814, 2)

In [22]:
preds1[:5,:]

array([[9.9965930e-01, 3.4070498e-04],
       [9.9974734e-01, 2.5264855e-04],
       [9.9993873e-01, 6.1231673e-05],
       [9.9998140e-01, 1.8655181e-05],
       [9.9801034e-01, 1.9896908e-03]], dtype=float32)

In [18]:
np.sum(tts_df.iloc[:,0])

17

In [25]:
tf_df['cp_type'] = tf_df['cp_type'].astype('category')
tf_df['cp_dose'] = tf_df['cp_dose'].astype('category')

In [30]:
# rerun this after setting loss & metric to log_loss
ak_pass1 = ak.StructuredDataClassifier(overwrite=True,max_trials=20,loss='binary_crossentropy',
                                       metrics=['binary_crossentropy','accuracy'])
ak_pass1.fit(tf_df,tts_df,epochs=20)

Trial 20 Complete [00h 09m 08s]
val_accuracy: 0.10059046745300293

Best val_accuracy So Far: 0.11577393859624863
Total elapsed time: 02h 12m 57s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
745/745 [==============================] - 22s 29ms/step - loss: 3.6895 - binary_crossentropy: 0.0228 - accuracy: 0.0166
Epoch 2/20
745/745 [==============================] - 23s 30ms/step - loss: 3.5540 - binary_crossentropy: 0.0221 - accuracy: 0.0305
Epoch 3/20
745/745 [==============================] - 23s 31ms/step - loss: 3.4766 - binary_crossentropy: 0.0217 - accuracy: 0.0408
Epoch 4/20
745/745 [==============================] - 26s 35ms/step - loss: 3.3954 - binary_crossentropy: 0.0213 - accuracy: 0.0544
Epoch 5/20
745/745 [==============================] - 23s 31ms/step - loss: 3.3316 - binary_crossentropy: 0.0209 - accuracy: 0.0645
Epoch 6/20
745/745 [==============================] - 23s 31ms/step - loss: 3.2787 - binary_crossentropy: 0.0207 - accuracy: 0.0695
Epoch 7/20
745/745 [=====

In [31]:
preds = ak_pass1.predict(tf_df)
metrics.log_loss(tts_df,preds)

19.099737395842038

In [32]:
ak_pass1.evaluate(tf_df,tts_df)

745/745 [==============================] - 14s 19ms/step - loss: 2.7565 - binary_crossentropy: 0.0179 - accuracy: 0.1334


[2.756510019302368, 0.017895421013236046, 0.1333669275045395]

In [33]:
preds = ak_pass1.predict(tf_df)
metrics.log_loss(tts_df,preds)

19.099737395842038

In [35]:
with open('ak-pass1.pkl','wb') as cellar:
    pickle.dump(ak_pass1,cellar)

In [36]:
keras1 = ak_pass1.export_model()

In [37]:
keras1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 875)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 875)               0         
_________________________________________________________________
normalization (Normalization (None, 875)               1751      
_________________________________________________________________
dense (Dense)                (None, 512)               448512    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)               

In [38]:
with open('ak-pass1-keras.pkl','wb') as cellar:
    pickle.dump(keras1,cellar)

TypeError: cannot pickle '_thread.RLock' object